<a href="https://colab.research.google.com/github/udaycharanrada/2d-to-pixar/blob/main/2D_to_Pixar_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U diffusers==0.26.3 huggingface_hub==0.25.2 transformers accelerate safetensors torch torchvision pillow ipywidgets


  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached pillow-12.0.0-cp311-cp311-win_amd64.whl.metadata (9.0 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
  Using cached transformers-4.56.1-py3-none-any.whl.metadata (42 kB)
  Using cached transformers-4.56.0-py3-none-any.whl.metadata (40 kB)
  Using cached transformers-4.55.4-py3-none-any.whl.metadata (41 kB)
  Using cached transformers-4.55.3-py3-none-any.whl.metadata (41 kB)
  Using cached transformers-4.55.2-py3-none-any.whl.metadata (41 kB)
  Using cached transformers-4.55.1-py3-none-any.whl.metadata (41 kB)
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-4.55.0-py3-none-any.whl.metadata (39 kB)
  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.14 requires protobuf<5,>=4.25.3, but you have protobuf 6.33.0 which is incompatible.
moviepy 2.2.1 requires pillow<12.0,>=9.2.0, but you have pillow 12.0.0 which is incompatible.
streamlit 1.50.0 requires pillow<12,>=7.1.0, but you have pillow 12.0.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\uday charan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from diffusers import StableDiffusionXLImg2ImgPipeline
import torch, random, os
from PIL import Image
import numpy as np, cv2
from IPython.display import display
import ipywidgets as widgets

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True
).to(device)

pipe.enable_attention_slicing()
pipe.enable_vae_tiling()

prompt = (
    "Pixar-style 3D render of a cartoon boy, perfect natural hands with five fingers, "
    "correct anatomy, detailed eyes, realistic proportions, cinematic lighting, smooth face, "
    "Pixar/Disney movie style, ultra-detailed, soft shadows, professional 3D shading"
)

negative_prompt = (
    "extra fingers, missing fingers, deformed hands, bad anatomy, distorted face, "
    "mutated limbs, low-res, blurry, watermark, text, creepy eyes, doll-like plastic texture"
)

# ---- FIX FOR LOCAL SYSTEM ----
image_path = input("Enter the path of your input image: ").strip()
init_image = Image.open(image_path).convert("RGB")

# Generate 5 outputs
os.makedirs("pixar_safe_variations", exist_ok=True)
files_list = []

for i in range(5):
    print(f"🎨 Generating Pixar render {i+1}/5 (safe mode)...")
    image = pipe(
        prompt=prompt,
        image=init_image,
        strength=0.65,
        guidance_scale=12,
        num_inference_steps=90,
        negative_prompt=negative_prompt,
        generator=torch.Generator(device).manual_seed(random.randint(0, 999999))
    ).images[0]

    image = pipe(
        prompt=prompt,
        image=image,
        strength=0.3,
        guidance_scale=9,
        num_inference_steps=40,
        negative_prompt=negative_prompt,
    ).images[0]

    fname = f"pixar_safe_variations/pixar_safe_{i+1}.png"
    image.save(fname)
    files_list.append(fname)

print("All 5 Pixar images generated successfully!")

# Scoring
def score_image(img):
    img_np = np.array(img.convert("RGB"))
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()
    contrast = img_np.std()
    return sharpness * 0.6 + contrast * 0.4

scores = [score_image(Image.open(f)) for f in files_list]
best_idx = int(np.argmax(scores))

print(f"Best image: {files_list[best_idx]} (score {scores[best_idx]:.2f})")


C:\Users\uday charan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

C:\Users\uday charan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\uday charan\.cache\huggingface\hub\models--stabilityai--stable-diffusion-xl-base-1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(mess

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

In [ ]:
# ============================================================
# 🎞 2D → 3D Parallax Motion Generator (Fixed Image Type Error)
# ============================================================

!pip install torch torchvision opencv-python moviepy --quiet

import torch, cv2, numpy as np
from PIL import Image
from moviepy.editor import ImageSequenceClip
from google.colab import files
import torchvision.transforms as T
import os

# ----------------------------------------
# 1️⃣ Upload your 2D image
# ----------------------------------------
uploaded = files.upload()
filename = next(iter(uploaded))
img = Image.open(filename).convert("RGB")
img_np = np.array(img)
H, W = img_np.shape[:2]
print(f"✅ Image loaded: {filename}, size = {W}x{H}")

# ----------------------------------------
# 2️⃣ Load MiDaS depth-estimation model (Torch Hub)
# ----------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Large").to(device).eval()

# Get transforms for DPT model
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

# ----------------------------------------
# 3️⃣ Run depth prediction (Fix: convert PIL → NumPy first)
# ----------------------------------------
# MiDaS expects NumPy array
img_cv = np.array(img)[:, :, ::-1]  # convert RGB → BGR for MiDaS
input_batch = transform(img_cv).to(device)

with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=(H, W),
        mode="bicubic",
        align_corners=False,
    ).squeeze()

depth = prediction.cpu().numpy()
depth_norm = (depth - depth.min()) / (depth.max() - depth.min() + 1e-8)
print("✅ Depth map computed successfully")

# ----------------------------------------
# 4️⃣ Generate parallax motion frames
# ----------------------------------------
n_frames = 75
motion_scale = 0.05
frames = []

for i in range(n_frames):
    t = i / (n_frames - 1)
    shift_x = int((t - 0.5) * motion_scale * W)
    shift_y = int((0.5 - t) * motion_scale * H)

    flow_x = depth_norm * shift_x
    flow_y = depth_norm * shift_y
    map_x, map_y = np.meshgrid(np.arange(W), np.arange(H))
    map_x = (map_x + flow_x).astype(np.float32)
    map_y = (map_y + flow_y).astype(np.float32)

    warped = cv2.remap(
        img_np, map_x, map_y,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT
    )
    frames.append(cv2.cvtColor(warped, cv2.COLOR_RGB2BGR))

print("✅ Generated parallax frames successfully")

# ----------------------------------------
# 5️⃣ Export as 5-second MP4 video
# ----------------------------------------
fps = int(n_frames / 5)
video_path = "parallax_3d_motion.mp4"
clip = ImageSequenceClip(frames, fps=fps)
clip.write_videofile(video_path, codec="libx264", audio=False)
files.download(video_path)
print("🎥 3D parallax video ready:", video_path)


Saving pixar_safe_4.png to pixar_safe_4.png
✅ Image loaded: pixar_safe_4.png, size = 600x648


  warnings.warn(



Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip


  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)



Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to /root/.cache/torch/hub/checkpoints/dpt_large_384.pt


...

✅ Depth map computed successfully
✅ Generated parallax frames successfully
Moviepy - Building video parallax_3d_motion.mp4.
Moviepy - Writing video parallax_3d_motion.mp4



Moviepy - Done !
Moviepy - video ready parallax_3d_motion.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎥 3D parallax video ready: parallax_3d_motion.mp4
